Notebook to create tables for paper:

- input: energy medians tables, execution time tables

In [3]:
import os
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg
import scikit_posthocs as sp
import seaborn as sns
#from cliffs_delta import cliffs_delta
from scipy import stats



In [4]:
pwd

'D:\\GAISSA\\energy-repo\\repo_sept_last\\energy-ml-serving\\experiments'

In [7]:
tables_cpuep_dir = 'D:/GAISSA/energy-repo/repo_sept_last/cpuep_processed/'
tables_cudaep_dir = 'D:/GAISSA/energy-repo/repo_sept_last/cudaep_processed/'

variables =['global_energy',
                "avg_cpu_usage_config",'avg_used_memory_pct_config',
                'avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',
                'avg_Load', 'duration'
                ]
dependent_variable = 'avg_utilization_gpu_config'

In [17]:
def save_latex_table(df,save_dir):
    df = df.copy()
    df.columns = [col.replace('_', ' ') for col in df.columns]
    
    latex_table = df.to_latex(index=False)

    # Define the filename
    filename = save_dir
    
    # Open the file in write mode
    with open(filename, 'w') as file:
        file.write(latex_table)
        
    # Print LaTeX table
    print(latex_table)

In [16]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [8]:
cpu_ep_data_energy = pd.read_csv(tables_cpuep_dir + "tables/median_energy_values.csv", index_col=None, header=0)
cuda_ep_data_energy = pd.read_csv(tables_cudaep_dir + "tables/median_energy_values.csv", index_col=None, header=0)

cpu_ep_data_time = pd.read_csv(tables_cpuep_dir + "tables/median_time_inference.csv", index_col=None, header=0)
cuda_ep_data_time = pd.read_csv(tables_cudaep_dir + "tables/median_time_inference.csv", index_col=None, header=0)

In [43]:
#cpu_ep_data_energy

## Energy

In [26]:
cpu_ep_data_energy['config'] = "cpuep_"+cpu_ep_data_energy['label']
cuda_ep_data_energy['config'] = "cudaep_"+cuda_ep_data_energy['label']

cuda_ep_data_energy

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
0,idle,idle,idle_idle,5.5,5331.372978,0.000000,5331.372978,cudaep_idle_idle
1,onnx,codeparrot-small,onnx_codeparrot-small,5.5,9081.845177,13986.934956,23077.985648,cudaep_onnx_codeparrot-small
2,onnx,phi2,onnx_phi2,5.5,40885.409996,87022.120454,127932.914739,cudaep_onnx_phi2
3,onnx,pythia-410m,onnx_pythia-410m,5.5,21988.519135,33065.796229,55228.025927,cudaep_onnx_pythia-410m
4,onnx,pythia1-4b,onnx_pythia1-4b,5.5,26553.710274,59124.014519,85651.403900,cudaep_onnx_pythia1-4b
5,onnx,tinyllama,onnx_tinyllama,5.5,20832.141495,46703.842778,67800.167767,cudaep_onnx_tinyllama
6,torch,codeparrot-small,torch_codeparrot-small,5.5,4424.862007,8426.396317,12825.179010,cudaep_torch_codeparrot-small
7,torch,phi2,torch_phi2,5.5,12209.897324,51308.245523,63554.592497,cudaep_torch_phi2
8,torch,pythia-410m,torch_pythia-410m,5.5,8847.555695,17862.435892,26693.627340,cudaep_torch_pythia-410m
9,torch,pythia1-4b,torch_pythia1-4b,5.5,9833.124313,31640.409523,41433.656472,cudaep_torch_pythia1-4b


In [27]:
energy_df = pd.concat([cpu_ep_data_energy,cuda_ep_data_energy], axis=0, ignore_index=True)
energy_df

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
0,idle,idle,idle_idle,5.5,5450.420303,0.000000,5450.420303,cpuep_idle_idle
1,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,7695.837711,35024.957307,cpuep_onnx_codeparrot-small
2,onnx,phi2,onnx_phi2,5.5,475452.949699,84742.714270,560611.319751,cpuep_onnx_phi2
3,onnx,pythia-410m,onnx_pythia-410m,5.5,75764.569603,20115.917003,95902.225445,cpuep_onnx_pythia-410m
4,onnx,pythia1-4b,onnx_pythia1-4b,5.5,212320.958244,49335.967747,261702.503859,cpuep_onnx_pythia1-4b
5,onnx,tinyllama,onnx_tinyllama,5.5,221268.890221,38324.805405,259593.695626,cpuep_onnx_tinyllama
6,ov,codeparrot-small,ov_codeparrot-small,5.5,24936.388664,5455.467005,30393.930993,cpuep_ov_codeparrot-small
7,ov,phi2,ov_phi2,5.5,442486.860435,45647.618833,488261.501346,cpuep_ov_phi2
8,ov,pythia-410m,ov_pythia-410m,5.5,125089.230072,20033.277757,145139.004625,cpuep_ov_pythia-410m
9,ov,pythia1-4b,ov_pythia1-4b,5.5,298005.695007,32518.830688,330258.365849,cpuep_ov_pythia1-4b


In [28]:
condition = energy_df['config'].str.endswith('idle')
energy_df = remove_condition_rows( condition, energy_df)
energy_df


,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
1,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,7695.837711,35024.957307,cpuep_onnx_codeparrot-small
2,onnx,phi2,onnx_phi2,5.5,475452.949699,84742.714270,560611.319751,cpuep_onnx_phi2
3,onnx,pythia-410m,onnx_pythia-410m,5.5,75764.569603,20115.917003,95902.225445,cpuep_onnx_pythia-410m
4,onnx,pythia1-4b,onnx_pythia1-4b,5.5,212320.958244,49335.967747,261702.503859,cpuep_onnx_pythia1-4b
5,onnx,tinyllama,onnx_tinyllama,5.5,221268.890221,38324.805405,259593.695626,cpuep_onnx_tinyllama
6,ov,codeparrot-small,ov_codeparrot-small,5.5,24936.388664,5455.467005,30393.930993,cpuep_ov_codeparrot-small
7,ov,phi2,ov_phi2,5.5,442486.860435,45647.618833,488261.501346,cpuep_ov_phi2
8,ov,pythia-410m,ov_pythia-410m,5.5,125089.230072,20033.277757,145139.004625,cpuep_ov_pythia-410m
9,ov,pythia1-4b,ov_pythia1-4b,5.5,298005.695007,32518.830688,330258.365849,cpuep_ov_pythia1-4b
10,ov,tinyllama,ov_tinyllama,5.5,257276.523949,28813.319413,286127.575814,cpuep_ov_tinyllama


In [29]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'CPU_energy', 'GPU_energy', 'global_energy']
energy_df_to_save = energy_df.sort_values(by=dependent_variable, ascending=True)[new_columns]
energy_df_to_save

,model,config,CPU_energy,GPU_energy,global_energy
1,codeparrot-small,cpuep_onnx_codeparrot-small,27370.594078,7695.837711,35024.957307
6,codeparrot-small,cpuep_ov_codeparrot-small,24936.388664,5455.467005,30393.930993
11,codeparrot-small,cpuep_torch_codeparrot-small,38922.891640,9092.602461,48006.204243
16,codeparrot-small,cpuep_torchscript_codeparrot-small,3198.062904,1829.733874,5030.619447
22,codeparrot-small,cudaep_onnx_codeparrot-small,9081.845177,13986.934956,23077.985648
27,codeparrot-small,cudaep_torch_codeparrot-small,4424.862007,8426.396317,12825.179010
32,codeparrot-small,cudaep_torchscript_codeparrot-small,1900.725342,5820.772408,7706.116044
2,phi2,cpuep_onnx_phi2,475452.949699,84742.714270,560611.319751
7,phi2,cpuep_ov_phi2,442486.860435,45647.618833,488261.501346
12,phi2,cpuep_torch_phi2,494766.105080,114590.373437,609310.464135


## Execution time

In [31]:
cpu_ep_data_time['config'] = "cpuep_"+cpu_ep_data_time['label']
cuda_ep_data_time['config'] = "cudaep_"+cuda_ep_data_time['label']

cuda_ep_data_time

,runtime,model,label,duration,config
0,onnx,codeparrot-small,onnx_codeparrot-small,191.048335,cudaep_onnx_codeparrot-small
1,onnx,phi2,onnx_phi2,793.478886,cudaep_onnx_phi2
2,onnx,pythia-410m,onnx_pythia-410m,447.527716,cudaep_onnx_pythia-410m
3,onnx,pythia1-4b,onnx_pythia1-4b,535.354030,cudaep_onnx_pythia1-4b
4,onnx,tinyllama,onnx_tinyllama,401.162165,cudaep_onnx_tinyllama
5,torch,codeparrot-small,torch_codeparrot-small,94.684550,cudaep_torch_codeparrot-small
6,torch,phi2,torch_phi2,251.969879,cudaep_torch_phi2
7,torch,pythia-410m,torch_pythia-410m,189.228687,cudaep_torch_pythia-410m
8,torch,pythia1-4b,torch_pythia1-4b,190.243684,cudaep_torch_pythia1-4b
9,torch,tinyllama,torch_tinyllama,188.817668,cudaep_torch_tinyllama


In [32]:
time_df = pd.concat([cpu_ep_data_time,cuda_ep_data_time], axis=0, ignore_index=True)
time_df

,runtime,model,label,duration,config
0,onnx,codeparrot-small,onnx_codeparrot-small,400.491458,cpuep_onnx_codeparrot-small
1,onnx,phi2,onnx_phi2,4454.560860,cpuep_onnx_phi2
2,onnx,pythia-410m,onnx_pythia-410m,1051.423814,cpuep_onnx_pythia-410m
3,onnx,pythia1-4b,onnx_pythia1-4b,2570.591944,cpuep_onnx_pythia1-4b
4,onnx,tinyllama,onnx_tinyllama,2000.845366,cpuep_onnx_tinyllama
5,ov,codeparrot-small,ov_codeparrot-small,285.900419,cpuep_ov_codeparrot-small
6,ov,phi2,ov_phi2,2393.867481,cpuep_ov_phi2
7,ov,pythia-410m,ov_pythia-410m,1032.987605,cpuep_ov_pythia-410m
8,ov,pythia1-4b,ov_pythia1-4b,1705.546433,cpuep_ov_pythia1-4b
9,ov,tinyllama,ov_tinyllama,1500.745790,cpuep_ov_tinyllama


In [33]:
condition = time_df['config'].str.endswith('idle')
time_df = remove_condition_rows( condition, time_df)

In [36]:
dependent_variable = ['model', 'config']
new_columns = ['model', 'config', 'duration']
time_df_to_save = time_df.sort_values(by=dependent_variable, ascending=True)[new_columns]
time_df_to_save

,model,config,duration
0,codeparrot-small,cpuep_onnx_codeparrot-small,400.491458
5,codeparrot-small,cpuep_ov_codeparrot-small,285.900419
10,codeparrot-small,cpuep_torch_codeparrot-small,472.177037
15,codeparrot-small,cpuep_torchscript_codeparrot-small,94.470121
20,codeparrot-small,cudaep_onnx_codeparrot-small,191.048335
25,codeparrot-small,cudaep_torch_codeparrot-small,94.684550
30,codeparrot-small,cudaep_torchscript_codeparrot-small,94.751527
1,phi2,cpuep_onnx_phi2,4454.560860
6,phi2,cpuep_ov_phi2,2393.867481
11,phi2,cpuep_torch_phi2,5983.522681


In [40]:
energy_df.join(time_df.set_index('config'), on='config', how='left', lsuffix='_left', rsuffix='_right')

,runtime_left,model_left,label_left,experiment,CPU_energy,GPU_energy,global_energy,config,runtime_right,model_right,label_right,duration
1,onnx,codeparrot-small,onnx_codeparrot-small,5.5,27370.594078,7695.837711,35024.957307,cpuep_onnx_codeparrot-small,onnx,codeparrot-small,onnx_codeparrot-small,400.491458
2,onnx,phi2,onnx_phi2,5.5,475452.949699,84742.714270,560611.319751,cpuep_onnx_phi2,onnx,phi2,onnx_phi2,4454.560860
3,onnx,pythia-410m,onnx_pythia-410m,5.5,75764.569603,20115.917003,95902.225445,cpuep_onnx_pythia-410m,onnx,pythia-410m,onnx_pythia-410m,1051.423814
4,onnx,pythia1-4b,onnx_pythia1-4b,5.5,212320.958244,49335.967747,261702.503859,cpuep_onnx_pythia1-4b,onnx,pythia1-4b,onnx_pythia1-4b,2570.591944
5,onnx,tinyllama,onnx_tinyllama,5.5,221268.890221,38324.805405,259593.695626,cpuep_onnx_tinyllama,onnx,tinyllama,onnx_tinyllama,2000.845366
6,ov,codeparrot-small,ov_codeparrot-small,5.5,24936.388664,5455.467005,30393.930993,cpuep_ov_codeparrot-small,ov,codeparrot-small,ov_codeparrot-small,285.900419
7,ov,phi2,ov_phi2,5.5,442486.860435,45647.618833,488261.501346,cpuep_ov_phi2,ov,phi2,ov_phi2,2393.867481
8,ov,pythia-410m,ov_pythia-410m,5.5,125089.230072,20033.277757,145139.004625,cpuep_ov_pythia-410m,ov,pythia-410m,ov_pythia-410m,1032.987605
9,ov,pythia1-4b,ov_pythia1-4b,5.5,298005.695007,32518.830688,330258.365849,cpuep_ov_pythia1-4b,ov,pythia1-4b,ov_pythia1-4b,1705.546433
10,ov,tinyllama,ov_tinyllama,5.5,257276.523949,28813.319413,286127.575814,cpuep_ov_tinyllama,ov,tinyllama,ov_tinyllama,1500.745790


In [64]:
df_to_save = energy_df.join(time_df.set_index('config'), on='config', how='left', lsuffix='_left', rsuffix='_right')
df_to_save.columns

Index(['runtime_left', 'model_left', 'label_left', 'experiment', 'CPU_energy',
       'GPU_energy', 'global_energy', 'config', 'runtime_right', 'model_right',
       'label_right', 'duration'],
      dtype='object')

In [65]:
columns_to_save  = ['model_left', 'config', 'CPU_energy','GPU_energy', 'global_energy', 'duration']

df_to_save = df_to_save[columns_to_save]
df_to_save.sort_values(by=['model_left','config'], ascending=True)

,model_left,config,CPU_energy,GPU_energy,global_energy,duration
1,codeparrot-small,cpuep_onnx_codeparrot-small,27370.594078,7695.837711,35024.957307,400.491458
6,codeparrot-small,cpuep_ov_codeparrot-small,24936.388664,5455.467005,30393.930993,285.900419
11,codeparrot-small,cpuep_torch_codeparrot-small,38922.891640,9092.602461,48006.204243,472.177037
16,codeparrot-small,cpuep_torchscript_codeparrot-small,3198.062904,1829.733874,5030.619447,94.470121
22,codeparrot-small,cudaep_onnx_codeparrot-small,9081.845177,13986.934956,23077.985648,191.048335
27,codeparrot-small,cudaep_torch_codeparrot-small,4424.862007,8426.396317,12825.179010,94.684550
32,codeparrot-small,cudaep_torchscript_codeparrot-small,1900.725342,5820.772408,7706.116044,94.751527
2,phi2,cpuep_onnx_phi2,475452.949699,84742.714270,560611.319751,4454.560860
7,phi2,cpuep_ov_phi2,442486.860435,45647.618833,488261.501346,2393.867481
12,phi2,cpuep_torch_phi2,494766.105080,114590.373437,609310.464135,5983.522681


In [66]:
df_to_save['config'].unique()

array(['cpuep_onnx_codeparrot-small', 'cpuep_onnx_phi2',
       'cpuep_onnx_pythia-410m', 'cpuep_onnx_pythia1-4b',
       'cpuep_onnx_tinyllama', 'cpuep_ov_codeparrot-small',
       'cpuep_ov_phi2', 'cpuep_ov_pythia-410m', 'cpuep_ov_pythia1-4b',
       'cpuep_ov_tinyllama', 'cpuep_torch_codeparrot-small',
       'cpuep_torch_phi2', 'cpuep_torch_pythia-410m',
       'cpuep_torch_pythia1-4b', 'cpuep_torch_tinyllama',
       'cpuep_torchscript_codeparrot-small', 'cpuep_torchscript_phi2',
       'cpuep_torchscript_pythia-410m', 'cpuep_torchscript_pythia1-4b',
       'cpuep_torchscript_tinyllama', 'cudaep_onnx_codeparrot-small',
       'cudaep_onnx_phi2', 'cudaep_onnx_pythia-410m',
       'cudaep_onnx_pythia1-4b', 'cudaep_onnx_tinyllama',
       'cudaep_torch_codeparrot-small', 'cudaep_torch_phi2',
       'cudaep_torch_pythia-410m', 'cudaep_torch_pythia1-4b',
       'cudaep_torch_tinyllama', 'cudaep_torchscript_codeparrot-small',
       'cudaep_torchscript_phi2', 'cudaep_torchscript_pythia-

In [67]:
def remove_after_second_underscore(value):
    # Split the string by '_', take only the first three parts, and join them back with '_'
    return '_'.join(value.split('_')[:2])

In [68]:
df_to_save['config'] = df_to_save['config'].apply(remove_after_second_underscore)
df_to_save

,model_left,config,CPU_energy,GPU_energy,global_energy,duration
1,codeparrot-small,cpuep_onnx,27370.594078,7695.837711,35024.957307,400.491458
2,phi2,cpuep_onnx,475452.949699,84742.714270,560611.319751,4454.560860
3,pythia-410m,cpuep_onnx,75764.569603,20115.917003,95902.225445,1051.423814
4,pythia1-4b,cpuep_onnx,212320.958244,49335.967747,261702.503859,2570.591944
5,tinyllama,cpuep_onnx,221268.890221,38324.805405,259593.695626,2000.845366
6,codeparrot-small,cpuep_ov,24936.388664,5455.467005,30393.930993,285.900419
7,phi2,cpuep_ov,442486.860435,45647.618833,488261.501346,2393.867481
8,pythia-410m,cpuep_ov,125089.230072,20033.277757,145139.004625,1032.987605
9,pythia1-4b,cpuep_ov,298005.695007,32518.830688,330258.365849,1705.546433
10,tinyllama,cpuep_ov,257276.523949,28813.319413,286127.575814,1500.745790


In [69]:
df_to_save['config'].unique()

array(['cpuep_onnx', 'cpuep_ov', 'cpuep_torch', 'cpuep_torchscript',
       'cudaep_onnx', 'cudaep_torch', 'cudaep_torchscript'], dtype=object)

In [70]:
# Mapping dictionary
#mapping = {'cpuep_onnx': 'C1', 'cpuep_ov': 'C2', 'cpuep_torch': 'C3','cpuep_torchscript': 'C4','cudaep_onnx': 'C5', 'cudaep_torch': 'C6', 'cudaep_torchscript': 'C7',}
mapping = {'cpuep_onnx': '<CPU, ONNX>', 'cpuep_ov': '<CPU, OV>', 'cpuep_torch': '<CPU, TORCH>','cpuep_torchscript': '<CPU, JIT>',
          'cudaep_onnx': '<CUDA, ONNX>', 'cudaep_torch': '<CUDA, TORCH>', 'cudaep_torchscript': '<CUDA, JIT>',}

# Apply mapping to the 'category' column
df_to_save['config'] = df_to_save['config'].map(mapping)
df_to_save = df_to_save.rename(columns={'model_left': 'model'})
df_to_save.sort_values(by=['model','config'], ascending=True)

,model,config,CPU_energy,GPU_energy,global_energy,duration
16,codeparrot-small,"<CPU, JIT>",3198.062904,1829.733874,5030.619447,94.470121
1,codeparrot-small,"<CPU, ONNX>",27370.594078,7695.837711,35024.957307,400.491458
6,codeparrot-small,"<CPU, OV>",24936.388664,5455.467005,30393.930993,285.900419
11,codeparrot-small,"<CPU, TORCH>",38922.891640,9092.602461,48006.204243,472.177037
32,codeparrot-small,"<CUDA, JIT>",1900.725342,5820.772408,7706.116044,94.751527
22,codeparrot-small,"<CUDA, ONNX>",9081.845177,13986.934956,23077.985648,191.048335
27,codeparrot-small,"<CUDA, TORCH>",4424.862007,8426.396317,12825.179010,94.684550
17,phi2,"<CPU, JIT>",25242.733498,3836.562885,29056.608449,189.110978
2,phi2,"<CPU, ONNX>",475452.949699,84742.714270,560611.319751,4454.560860
7,phi2,"<CPU, OV>",442486.860435,45647.618833,488261.501346,2393.867481


In [71]:
pwd

'D:\\GAISSA\\energy-repo\\repo_sept_last\\energy-ml-serving\\experiments'

In [72]:
save_latex_table(df_to_save.sort_values(by=['model','config'], ascending=True) , 'D:/GAISSA/energy-repo/repo_sept_last/overleaf_latex_tables/energy_time_results.tex')

\begin{tabular}{llrrrr}
\toprule
model & config & CPU energy & GPU energy & global energy & duration \\
\midrule
codeparrot-small & <CPU, JIT> & 3198.062904 & 1829.733874 & 5030.619447 & 94.470121 \\
codeparrot-small & <CPU, ONNX> & 27370.594078 & 7695.837711 & 35024.957307 & 400.491458 \\
codeparrot-small & <CPU, OV> & 24936.388664 & 5455.467005 & 30393.930993 & 285.900419 \\
codeparrot-small & <CPU, TORCH> & 38922.891640 & 9092.602461 & 48006.204243 & 472.177037 \\
codeparrot-small & <CUDA, JIT> & 1900.725342 & 5820.772408 & 7706.116044 & 94.751527 \\
codeparrot-small & <CUDA, ONNX> & 9081.845177 & 13986.934956 & 23077.985648 & 191.048335 \\
codeparrot-small & <CUDA, TORCH> & 4424.862007 & 8426.396317 & 12825.179010 & 94.684550 \\
phi2 & <CPU, JIT> & 25242.733498 & 3836.562885 & 29056.608449 & 189.110978 \\
phi2 & <CPU, ONNX> & 475452.949699 & 84742.714270 & 560611.319751 & 4454.560860 \\
phi2 & <CPU, OV> & 442486.860435 & 45647.618833 & 488261.501346 & 2393.867481 \\
phi2 & <CPU, TO

In [61]:
tables_cpuep_dir

'D:/GAISSA/energy-repo/repo_sept_last/cpuep_processed/'